In [1]:
import decoupler as dc

In [2]:
net = dc.get_collectri(organism='human', split_complexes=False)
net

  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\connectionpool.py", line 789, in urlopen
    response = self._make_request(
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\connectionpool.py", line 490, in _make_request
    raise new_e
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\connectionpool.py", line 466, in _make_request
    self._validate_conn(conn)
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\connectionpool.py", line 1095, in _validate_conn
    conn.connect()
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\connection.py", line 730, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
  File "c:\Users\20191678\AppData\Local\miniconda3\envs\5ARG45\lib\site-packages\urllib3\connection.py", line 909, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_

,source,target,weight,PMID
0,MYC,TERT,1,10022128;10491298;10606235;10637317;10723141;1...
1,SPI1,BGLAP,1,10022617
2,SMAD3,JUN,1,10022869;12374795
3,SMAD4,JUN,1,10022869;12374795
4,STAT5A,IL2,1,10022878;11435608;17182565;17911616;22854263;2...
...,...,...,...,...
43173,NFKB,hsa-miR-143-3p,1,19472311
43174,AP1,hsa-miR-206,1,19721712
43175,NFKB,hsa-miR-21-5p,1,20813833;22387281
43176,NFKB,hsa-miR-224-5p,1,23474441;23988648


In [13]:
net["source"].unique().__len__()

1186

In [6]:
import pandas as pd
import torch


def create_gene_tf_matrix(
    net: pd.DataFrame, genes: list = None, tfs: list = None
) -> torch.Tensor:
    """
    Creates a PyTorch tensor representing the gene-TF regulatory matrix from a network dataframe.

    Args:
        net (pd.DataFrame): DataFrame containing the regulatory network with the following columns:
            - "source": Transcription factors (TFs).
            - "target": Genes regulated by the TFs.
            - "weight": Interaction weight (1 for activation, -1 for inhibition).
        genes (list, optional): List of genes to include in the matrix. If None, all unique genes in `net` are used.
        tfs (list, optional): List of TFs to include in the matrix. If None, all unique TFs in `net` are used.

    Returns:
        torch.Tensor: A tensor of shape (num_genes, num_tfs) where:
            - `1` indicates an activating interaction.
            - `-1` indicates an inhibiting interaction.
            - `0` indicates no interaction.
    """
    # Validate input
    required_columns = {"source", "target", "weight"}
    if not required_columns.issubset(net.columns):
        raise ValueError(
            f"The `net` dataframe must contain the columns: {required_columns}"
        )

    # Use all unique genes and TFs if not provided
    if genes is None:
        genes = sorted(net["target"].unique())
    if tfs is None:
        tfs = sorted(net["source"].unique())

    # Initialize a DataFrame with zeros (default for no interaction)
    gene_to_tf_df = pd.DataFrame(0, index=genes, columns=tfs, dtype=float)

    # Populate the DataFrame with interaction weights
    for _, row in net.iterrows():
        gene = row["target"]
        tf = row["source"]
        weight = row["weight"]
        if gene in genes and tf in tfs:
            gene_to_tf_df.at[gene, tf] = weight

    # Convert the DataFrame to a PyTorch tensor
    gene_to_tf_matrix = torch.tensor(gene_to_tf_df.values, dtype=torch.float32)
    return gene_to_tf_matrix

In [7]:
matrix = create_gene_tf_matrix(net)
matrix

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [14]:
matrix.unique(return_counts=True)

(tensor([-1.,  0.,  1.]), tensor([   5854, 7893534,   37324]))